In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.pipeline import Pipeline 
dataset = pd.read_csv('../data/uci ai4i2020.csv')
features_to_keep = ['Type','Air temperature [K]','Process temperature [K]','Rotational speed [rpm]','Torque [Nm]','Tool wear [min]']
X = dataset[features_to_keep]
y = dataset['Machine failure'].values
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(
    transformers=[
        ("encoder", OneHotEncoder(drop="first"), ["Type","Product ID"])], remainder="passthrough"
)
X = ct.fit_transform(X)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=1)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
print("Train Accuracy:", model.score(X_train, y_train))
print("Test Accuracy:", model.score(X_test, y_test))
print(model.coef_)
print(model.intercept_)
import dice_ml
from dice_ml.utils import helpers
features_to_keep = ['Type', 'Air temperature [K]', 
                    'Process temperature [K]', 'Rotational speed [rpm]',
                     'Torque [Nm]', 'Tool wear [min]']
df_dice = dataset[features_to_keep + ['Machine failure']]
data_dice = dice_ml.Data(
    dataframe=df_dice,
    continuous_features=['Air temperature [K]',
                          'Process temperature [K]', 'Rotational speed [rpm]',
                          'Torque [Nm]', 'Tool wear [min]'],
    categorical_features=['Type'],
    outcome_name='Machine failure'
)
# 1. اربط الـ Encoder بالموديل (الـ Pipeline هو اللي بيفهم الحروف)
target_model = Pipeline(steps=[
    ('preprocessor', ct),
    ('classifier', model)
])

# 2. عرف الموديل لـ DiCE باستخدام الـ Pipeline (target_model)
model_di = dice_ml.Model(model=target_model, backend='sklearn')

# 3. عرف الـ Explainer (مرة واحدة فقط)
explainer = dice_ml.Dice(data_dice, model_di, method='random')
feature_names = ['Product ID', 
    'Type', 
    'Air temperature [K]', 
    'Process temperature [K]', 
    'Rotational speed [rpm]', 
    'Torque [Nm]', 
    'Tool wear [min]']
input_datapoint = dataset.loc[[5], features_to_keep]   
features_to_vary=['Air temperature [K]','Process temperature [K]','Tool wear [min]',
                  'Torque [Nm]','Rotational speed [rpm]']
counterfactuals = explainer.generate_counterfactuals(
         input_datapoint, 
         total_CFs=4, 
        desired_class="opposite",
        features_to_vary=features_to_vary,
         permitted_range={
             'Air temperature [K]': [0, 100], 'Process temperature [K]': [0, 100], 
             'Tool wear [min]': [0, 125],
               'Torque [Nm]': [0, 60], 
             'Rotational speed [rpm]': [0, 1000]}
            ,sample_size=10000)
#  #features_to_vary={'temperature','pressure','vibration',}
counterfactuals.visualize_as_dataframe(show_only_changes=True)


c:\Users\DELL\Desktop\bacholer\venv\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train Accuracy: 0.969
Test Accuracy: 0.9735
[[ 1.67905648e-01 -7.95733722e-02 -5.87707374e-05 ...  1.09947294e-02
   2.70489793e-01  1.26487500e-02]]
[-0.00707633]


  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: columns are missing: {'UDI', 'Product ID'}